In [2]:
import requests
from bs4 import BeautifulSoup
import re
import json

# URLs oficiais dos programas sociais
urls = {
    "Bolsa Família": "https://www.gov.br/mds/pt-br/acoes-e-programas/bolsa-familia",
    "BPC": "https://www.gov.br/mds/pt-br/acoes-e-programas/suas/beneficios-assistenciais/beneficio-assistencial-ao-idoso-e-a-pessoa-com-deficiencia-bpc"
}

# Função para extrair texto de uma página
def extrair_texto(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        return soup.get_text(separator=" ").lower()
    else:
        print(f"Erro ao acessar {url}: {response.status_code}")
        return ""

# Função para buscar critérios no texto
def extrair_criterios(programa, texto):
    criterios = {}

    if programa == "Bolsa Família":
        renda = re.search(r"r\$\s*([0-9]+)\s*,?\.?([0-9]*)", texto)
        if renda:
            criterios["renda_per_capita_max"] = float(renda.group(1))
        criterios["cad_unico_required"] = "cadastro único" in texto
        criterios["condicionalidades"] = []
        if "frequência escolar" in texto:
            criterios["condicionalidades"].append("frequência escolar")
        if "vacinação" in texto:
            criterios["condicionalidades"].append("vacinação")
        if "pré-natal" in texto:
            criterios["condicionalidades"].append("pré-natal")

    elif programa == "BPC":
        if "65" in texto:
            criterios["idade_min_idoso"] = 65
        if "1/4" in texto or "um quarto" in texto:
            criterios["renda_per_capita_max_frac_salario_minimo"] = 0.25
        criterios["cad_unico_required"] = "cadastro único" in texto
        if "aposentadoria" in texto or "pensão" in texto:
            criterios["no_acumulo"] = ["aposentadoria", "pensão"]

    return criterios

# Montando JSON de critérios
todos_criterios = {}
for programa, url in urls.items():
    texto = extrair_texto(url)
    criterios = extrair_criterios(programa, texto)
    todos_criterios[programa] = criterios

# Exibindo critérios em formato JSON
print(json.dumps(todos_criterios, indent=2, ensure_ascii=False))

{
  "Bolsa Família": {
    "renda_per_capita_max": 218.0,
    "cad_unico_required": true,
    "condicionalidades": []
  },
  "BPC": {
    "idade_min_idoso": 65,
    "renda_per_capita_max_frac_salario_minimo": 0.25,
    "cad_unico_required": true,
    "no_acumulo": [
      "aposentadoria",
      "pensão"
    ]
  }
}
